__BIOBSS - ECG Signal Processing__

_This notebook includes guidelines to help using BIOBSS for ECG signal processing and feature extraction._

In [1]:
#Import BIOBSS and the other required packages

import biobss
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#help(biobss)

# Table of Contents
1.[ECG Sample Data](#sampledata)<br>
2.[ECG Signal Filtering](#ecg_pre)<br>
3.[ECG Peak Detection](#ecg_peak)<br>
4.[ECG Delineation](#ecg_waves)<br>
5.[ECG Signal Quality Assessment](#ecg_sqa)<br>
6.[ECG Feature Extraction](#ecg_features)<br>

### __ECG Sample Data__
<a id="sampledata"></a>

In [2]:
#Load the sample data. The sample data file contains 100 10-s PPG segments. The samling rate of the segments is 64 Hz. 

data_dir=r'C:\Users\ipek.karakus\Desktop\biobss\sample_data'

filename='ecg_sample_data.csv'
fs=256
L=10
data=pd.read_csv(os.path.join(data_dir,filename))
sig=np.asarray(data.iloc[:fs*L,0]) #The first column is selected to be used as sample signal. 


### __ECG Signal Filtering__
<a id="ecg_pre"></a>

BIOBSS library provides a filtering function based on Scipy. This function can be used to implement Butterworth filter by defining the filter parameters.

In [ ]:
#or defining the parameters.

f_sig= biobss.preprocess.filter_signal(sig,fs,'bandpass',N=2,f_lower=0.5,f_upper=5)

Additionally, pre-defined filter parameters can be used for each specific signal type. In order to use this option, signal_type should be selected as 'ECG'. 

In [ ]:
#ECG signals can be filtered using the method below using pre-determined filter parameters

filtered_ecg=biobss.preprocess.filter_signal(sig, sampling_rate=fs, signal_type='ECG', method='elgendi')

### __ECG Peak Detection__
<a id="ecg_peak"></a>

In [3]:
#Signal peaks can be detected as below

locs_peaks=biobss.ecgtools.ecg_peaks(sig,fs,'pantompkins')
peaks = sig[locs_peaks]

### __ECG Delineation__
<a id="ecg_waves"></a>

In [4]:
_, fiducials = biobss.ecgtools.ecg_waves(sig, locs_peaks, fs, 'neurokit2')

p_peaks_locs = fiducials['ECG_P_Peaks']
q_peaks_locs = fiducials['ECG_Q_Peaks']
s_peaks_locs = fiducials['ECG_S_Peaks']
t_peaks_locs = fiducials['ECG_T_Peaks']
p_onset_locs = fiducials['ECG_P_Onsets']
t_offset_locs = fiducials['ECG_T_Offsets']

### __ECG Signal Quality Assessment__
<a id="ecg_sqa"></a>

In [ ]:
#Clipping and flatline detection can be done using the methods below
#For clipping detection
info=biobss.ppgtools.detect_flatline_clipping(sig,threshold=1.0, clipping=True)

In [ ]:
#For flatline detection
info=biobss.ppgtools.detect_flatline_clipping(sig,threshold=0.0001,flatline=True, duration=10)

In [ ]:
#The peaks should be detected for the following steps
locs_peaks=biobss.ecgtools.ecg_peaks(sig,fs,'pantompkins')
peaks = sig[locs_peaks]

In [ ]:
#Template matching method can be applied using the method below.

info=biobss.ppgtools.template_matching(sig,locs_peaks)

### __ECG Feature Extraction__
<a id="ecg_features"></a>

In [ ]:
#Features can be calculated using R-peak locations
biobss.ecgtools.ecg_features.from_Rpeaks(sig, locs_peaks, fs)

In [5]:
#or using all fiducial locations
biobss.ecgtools.ecg_features.from_waves(sig, locs_peaks, fiducials, fs)

{'m': {'ecg_t_PR': 0.15625,
  'ecg_t_QR': 0.08203125,
  'ecg_t_SR': 0.2734375,
  'ecg_t_TR': 0.3984375,
  'ecg_t_PQ': 0.07421875,
  'ecg_t_PS': 0.4296875,
  'ecg_t_PT': 0.5546875,
  'ecg_t_QS': 0.35546875,
  'ecg_t_QT': 0.48046875,
  'ecg_t_ST': 0.125,
  'ecg_t_PT_QS': 1.5604395604395604,
  'ecg_t_QT_QS': 1.3516483516483517,
  'ecg_a_PQ': -0.20500000000000002,
  'ecg_a_QR': -0.174,
  'ecg_a_RS': 0.159,
  'ecg_a_ST': 0.079,
  'ecg_a_PS': -0.22,
  'ecg_a_PT': -0.14100000000000001,
  'ecg_a_QS': -0.015,
  'ecg_a_QT': 0.064,
  'ecg_a_ST_QS': -5.266666666666667,
  'ecg_a_RS_QR': -0.9137931034482759,
  'ecg_a_PQ_QS': 13.666666666666668,
  'ecg_a_PQ_QT': -3.203125,
  'ecg_a_PQ_PS': 0.9318181818181819,
  'ecg_a_PQ_QR': 1.17816091954023,
  'ecg_a_PQ_RS': -1.289308176100629,
  'ecg_a_RS_QS': -10.600000000000001,
  'ecg_a_RS_QT': 2.484375,
  'ecg_a_ST_PQ': -0.38536585365853654,
  'ecg_a_ST_QT': 1.234375}}